## Topic Analysis of Review Data

#### DESCRIPTION

Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

#### Problem Statement: 

A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

**Domain:** Amazon reviews for a leading phone brand

#### Analysis to be done: 
POS tagging, topic modeling using LDA, and topic interpretation

**Dataset:** ‘K8 Reviews v0.2.csv’

**Sentiment:** The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)

**Reviews:** The main text of the review

#### Steps to perform:

Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.

Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

#### Tasks: 

- Read the .csv file using Pandas. Take a look at the top few records.

- Normalize casings for the review text and extract the text into a list for easier manipulation.

- Tokenize the reviews using NLTKs word_tokenize function.

- Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

- For the topic model, we should  want to include only nouns.

  - Find out all the POS tags that correspond to nouns.

  - Limit the data to only terms with these tags.

- Lemmatize. 

  - Different forms of the terms need to be treated as one.

  - No need to provide POS tag to lemmatizer for now.

- Remove stopwords and punctuation (if there are any). 

- Create a topic model using LDA on the cleaned up data with 12 topics.

  - Print out the top terms for each topic.

  - What is the coherence of the model with the c_v metric?

- Analyze the topics through the business lens.

  - Determine which of the topics can be combined.

- Create topic model using LDA with what you think is the optimal number of topics

  - What is the coherence of the model?

- The business should  be able to interpret the topics.

  - Name each of the identified topics.

  - Create a table with the topic name and the top 10 terms in each to present to the  business.

In [1]:
import pandas as pd
import numpy as np
import nltk

#### Read the data

In [2]:
data = pd.read_csv('K8 Reviews v0.2.csv')

In [3]:
data.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14675 entries, 0 to 14674
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  14675 non-null  int64 
 1   review     14675 non-null  object
dtypes: int64(1), object(1)
memory usage: 229.4+ KB


#### Normalizing the casing to lower

In [5]:
data['review_cleaned'] = data['review'].apply(lambda review: review.lower())

In [6]:
data.head()

,sentiment,review,review_cleaned
0,1,Good but need updates and improvements,good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr...","worst mobile i have bought ever, battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...,when i will get my 10% cash back.... its alrea...
3,1,Good,good
4,0,The worst phone everThey have changed the last...,the worst phone everthey have changed the last...


In [9]:
reviews = list(data['review_cleaned'])
reviews = [nltk.word_tokenize(review) 
           for review in reviews if review not in nltk.corpus.stopwords.words('english')]

In [11]:
len(reviews)

14674

In [12]:
import gensim
from gensim import corpora
from gensim import models

In [13]:
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(reviews)

In [15]:
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in reviews]

In [21]:
print("{} -> {}".format(data['review'][20],corpus[20]))

good -> [(2, 1)]


In [22]:
NUM_TOPICS = 12
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [23]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 5 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 5))

LDA Model:
Topic #0: 0.104*"not" + 0.073*"working" + 0.034*"is" + 0.029*"charger" + 0.027*"properly"
Topic #1: 0.092*"awesome" + 0.030*"phone" + 0.019*"bed" + 0.016*"cool" + 0.016*"missing"
Topic #2: 0.051*"." + 0.034*"it" + 0.029*"is" + 0.028*"phone" + 0.021*"this"
Topic #3: 0.052*"." + 0.052*"is" + 0.030*"not" + 0.029*"the" + 0.020*"it"
Topic #4: 0.047*"," + 0.040*"." + 0.030*"the" + 0.025*"is" + 0.025*"not"
Topic #5: 0.014*"phone.." + 0.014*"concern" + 0.013*"no" + 0.012*"is" + 0.011*"loved"
Topic #6: 0.049*"." + 0.035*"i" + 0.029*"the" + 0.026*"it" + 0.022*"this"
Topic #7: 0.123*"good" + 0.076*"is" + 0.062*"." + 0.053*"very" + 0.051*"phone"
Topic #8: 0.181*"..." + 0.054*"very" + 0.029*"is" + 0.027*"poor" + 0.024*"battery"
Topic #9: 0.198*"nice" + 0.161*"mobile" + 0.037*"phone" + 0.022*"?" + 0.010*"one"
Topic #10: 0.035*"do" + 0.032*"problem" + 0.032*"buy" + 0.031*"n't" + 0.029*"hai"
Topic #11: 0.113*"," + 0.050*"good" + 0.039*"camera" + 0.026*"!" + 0.023*"."


In [24]:
import pyLDAvis
import pyLDAvis.gensim 

In [25]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.162572  0.091367       1        1  27.227900
4     -0.149064  0.084473       2        1  19.847733
3     -0.159140  0.019916       3        1  16.699800
2     -0.123814  0.015589       4        1  10.138914
7     -0.098760 -0.163414       5        1   8.440128
11    -0.081702 -0.116654       6        1   5.512606
8     -0.064134 -0.055184       7        1   5.067953
10     0.131731  0.160128       8        1   2.367661
0      0.059613  0.055674       9        1   2.196792
9      0.209540 -0.090804      10        1   1.257189
1      0.260571 -0.022397      11        1   0.632504
5      0.177731  0.021306      12        1   0.610825, topic_info=       Term         Freq        Total Category  logprob  loglift
84      ...  4028.000000  4028.000000  Default  30.0000  30.0000
7         ,  6985.000000  6985.000000  Default  29.0000  29.0000
2      good  5896.000000  5896.000000  Default  28.0000  28.0000
63      not  6098.000000  6098.000000  Default  27.0000  27.0000
61   mobile  2326.000000  2326.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
226   there    10.534156   682.164917  Topic12  -5.2366   0.9275
81    worst     9.904473   888.866638  Topic12  -5.2983   0.6012
7         ,    10.059587  6985.592773  Topic12  -5.2827  -1.4450
0       and     9.586787  5538.837891  Topic12  -5.3309  -1.2610
75       to     8.338708  4803.469727  Topic12  -5.4703  -1.2581

[849 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
314       1  0.078829     !
314       2  0.034398     !
314       3  0.008600     !
314       4  0.008600     !
314       6  0.676499     !
...     ...       ...   ...
82        8  0.001731   you
82        9  0.001731   you
1080      4  0.989809  zoom
3006      6  0.948542     👍
2980     12  0.890381     🔋

[2416 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 4, 3, 8, 12, 9, 11, 1, 10, 2, 6])

In [ ]:
from collections import defaultdict
freq = defaultdict(int)

for review in reviews:
    for topic in review:
        freq[topic] += 1
len(freq)

In [ ]:
review_df = pd.DataFrame(zip(freq.keys(),freq.values()), columns=["topic","freq"])
review_df

In [ ]:
review_df['pos_tag'] = [tag[1] for tag in nltk.pos_tag(review_df['topic'])]
review_df.head()

In [ ]:
review_df_nouns = review_df[(review_df["pos_tag"].str.startswith('N')) & (review_df['topic'].str.len() > 5)]

In [ ]:
from nltk.stem import WordNetLemmatizer

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
review_df_nouns['topic'].apply(lambda x: lemmatizer.lemmatize(x))

In [ ]:
review_df_nouns.sort_values(by=['freq'], ascending=False).head(50)

In [ ]:
review_df_nouns.info()

In [ ]:
import gensim.corpora as corpora

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(review_df_nouns['topic'])